# Practical 11: Structural decomposition analysis

Objectives
- Construct different SDA methods in Python
- Reproduce the SDA examples on page 348-351 of Miller & Blair
- Reproduce output of table 8.1 on page 351 of Miller & Blair

In [1]:
# Import packages
import numpy as np
import pandas as pd

## Exercise 1: Construct the SDA

### 1.1 Define input data for baseline

In [2]:
# t=0 transaction matrix and final demand vector
Z_0 = pd.DataFrame([
    [10, 20, 25],
    [15, 5, 30],
    [30, 40, 5]
])

y_0 = pd.Series([45,30,25])

### 1.2 Define input data for delta

In [3]:
# t=1 transaction matrix and final demand vector

Z_1 = pd.DataFrame([
    [12, 15, 35],
    [24, 11, 30],
    [36, 50, 8]
])

y_1 = pd.Series([50,35,26])

### 1.3 Calculate total requirements matrix

In [4]:
# t=0 Baseline year

x_0 = Z_0.sum(axis=1) + y_0.values
A_0 = Z_0 /x_0.transpose()
L_0 = np.linalg.inv(np.eye(3) - A_0)  

In [5]:
# t=1 next year

x_1 = Z_1.sum(axis=1) + y_1
A_1 = Z_1 /x_1.transpose()
L_1 = np.linalg.inv(np.eye(3) - A_1)

### 1.4 Calculate your deltas for the decomposition

#### 1.4.1 Change in total requirement matrix

In [6]:
L_delta = L_1 - L_0

#### 1.4.2 Change in final demand

In [7]:
y_delta = y_1 - y_0

#### 1.4.3 Change in total output to be used as check

In [8]:
x_delta = x_1 - x_0

### 1.5 Decomposition

All input data for our SDA is now complete.

Start decomposing the effect of technology change and final demand change on the total output. 

A two factor decomposition.

#### 1.5.1 Decomposition (eq 8.3)

In [9]:
tech_change = L_delta @ y_0
y_change = L_1 @ y_delta

In [10]:
# Check if result is complete

tot_change = tech_change + y_change
check = x_delta - tot_change

if np.isclose(check, np.zeros(check.shape), atol=1E-6).all():
    print('Decomposition correct')
else:
    print('ERROR - check your decomposition')

Decomposition correct


In [11]:
# Organize the labels for the table in which you will store your results
sectors = ["Sector_1", "Sector_2", "Sector_3", "Total"] 
columns = ["Output_Change", "Technology_change_contribution", "Final_Demand_change_contribution", "Interaction_term"] 

# create a dataframe where you will store your intermediary results
step = pd.DataFrame([], columns=columns)

## add results to your dataframe
step[step.columns[1]] = tech_change
step[step.columns[2]] = y_change
step[step.columns[0]] = tot_change

## Add the row of total
step.loc["Total"] = step.sum(0)

step.index = [["Eq_8.3"]*4, sectors]

step

Output_Change  Technology_change_contribution  \
Eq_8.3 Sector_1           12.0                        0.898419   
       Sector_2           20.0                        8.622001   
       Sector_3           20.0                        9.009966   
       Total              52.0                       18.530386   

                 Final_Demand_change_contribution Interaction_term  
Eq_8.3 Sector_1                         11.101581              NaN  
       Sector_2                         11.377999              NaN  
       Sector_3                         10.990034              NaN  
       Total                            33.469614                0

In [12]:
# Make a copy of step that you will call results 
# and which you can use as a basis to assemble all results
results = step.copy()
results

Output_Change  Technology_change_contribution  \
Eq_8.3 Sector_1           12.0                        0.898419   
       Sector_2           20.0                        8.622001   
       Sector_3           20.0                        9.009966   
       Total              52.0                       18.530386   

                 Final_Demand_change_contribution Interaction_term  
Eq_8.3 Sector_1                         11.101581              NaN  
       Sector_2                         11.377999              NaN  
       Sector_3                         10.990034              NaN  
       Total                            33.469614                0

#### 1.5.2 Decomposition (eq 8.4)

In [13]:
tech_change = L_delta @ y_1
y_change = L_0 @ y_delta

In [14]:
# check if our result is complete
tot_change = tech_change + y_change

check = x_delta - tot_change

if np.isclose(check, np.zeros(check.shape), atol=1E-6).all():
    print('Decomposition correct')
else:
    print('ERROR - check your decomposition')


Decomposition correct


In [15]:
# create a dataframe where you will store your intermediary results
step = pd.DataFrame([], columns=columns)

## add results to your dataframe
step[step.columns[1]] = tech_change
step[step.columns[2]] = y_change
step[step.columns[0]] = tot_change

## Add the row of total
step.loc["Total"] = step.sum(0)

step.index = [["Eq_8.4"]*4, sectors]

step

Output_Change  Technology_change_contribution  \
Eq_8.4 Sector_1           12.0                        0.784597   
       Sector_2           20.0                        9.660650   
       Sector_3           20.0                        9.963899   
       Total              52.0                       20.409146   

                 Final_Demand_change_contribution Interaction_term  
Eq_8.4 Sector_1                         11.215403              NaN  
       Sector_2                         10.339350              NaN  
       Sector_3                         10.036101              NaN  
       Total                            31.590854                0

In [16]:
# Concatenate your results
results = pd.concat([results, step], axis=0)
results

Output_Change  Technology_change_contribution  \
Eq_8.3 Sector_1           12.0                        0.898419   
       Sector_2           20.0                        8.622001   
       Sector_3           20.0                        9.009966   
       Total              52.0                       18.530386   
Eq_8.4 Sector_1           12.0                        0.784597   
       Sector_2           20.0                        9.660650   
       Sector_3           20.0                        9.963899   
       Total              52.0                       20.409146   

                 Final_Demand_change_contribution Interaction_term  
Eq_8.3 Sector_1                         11.101581              NaN  
       Sector_2                         11.377999              NaN  
       Sector_3                         10.990034              NaN  
       Total                            33.469614                0  
Eq_8.4 Sector_1                         11.215403              NaN  
       Sector_2                         10.339350              NaN  
       Sector_3                         10.036101              NaN  
       Total                            31.590854                0

#### 1.5.3 Decomposition (eq 8.5)

In [17]:
tech_change = L_delta @ y_0
y_change = L_0 @ y_delta
interaction = L_delta @ y_delta

In [18]:
# check if our result is complete
tot_change = tech_change + y_change + interaction

check = x_delta - tot_change

if np.isclose(check, np.zeros(check.shape), atol=1E-6).all():
    print('Decomposition correct')
else:
    print('ERROR - check your decomposition')


Decomposition correct


In [19]:
# create a dataframe where you will store your intermediary results
step = pd.DataFrame([], columns=columns)

## add results to your dataframe
step[step.columns[1]] = tech_change
step[step.columns[2]] = y_change
step[step.columns[3]] = interaction
step[step.columns[0]] = tot_change

## Add the row of total
step.loc["Total"] = step.sum(0)

step.index = [["Eq_8.5"]*4, sectors]

step

Output_Change  Technology_change_contribution  \
Eq_8.5 Sector_1           12.0                        0.898419   
       Sector_2           20.0                        8.622001   
       Sector_3           20.0                        9.009966   
       Total              52.0                       18.530386   

                 Final_Demand_change_contribution  Interaction_term  
Eq_8.5 Sector_1                         11.215403         -0.113822  
       Sector_2                         10.339350          1.038648  
       Sector_3                         10.036101          0.953933  
       Total                            31.590854          1.878759

In [20]:
# Concatenate your results
results = pd.concat([results, step], axis=0)
results

Output_Change  Technology_change_contribution  \
Eq_8.3 Sector_1           12.0                        0.898419   
       Sector_2           20.0                        8.622001   
       Sector_3           20.0                        9.009966   
       Total              52.0                       18.530386   
Eq_8.4 Sector_1           12.0                        0.784597   
       Sector_2           20.0                        9.660650   
       Sector_3           20.0                        9.963899   
       Total              52.0                       20.409146   
Eq_8.5 Sector_1           12.0                        0.898419   
       Sector_2           20.0                        8.622001   
       Sector_3           20.0                        9.009966   
       Total              52.0                       18.530386   

                 Final_Demand_change_contribution Interaction_term  
Eq_8.3 Sector_1                         11.101581              NaN  
       Sector_2                         11.377999              NaN  
       Sector_3                         10.990034              NaN  
       Total                            33.469614                0  
Eq_8.4 Sector_1                         11.215403              NaN  
       Sector_2                         10.339350              NaN  
       Sector_3                         10.036101              NaN  
       Total                            31.590854                0  
Eq_8.5 Sector_1                         11.215403        -0.113822  
       Sector_2                         10.339350         1.038648  
       Sector_3                         10.036101         0.953933  
       Total                            31.590854         1.878759

#### 1.5.4 Decomposition (eq 8.6)

In [21]:
tech_change = L_delta @ y_1
y_change = L_1 @ y_delta
interaction = L_delta @ y_delta

In [22]:
# check if our result is complete
tot_change = tech_change + y_change - interaction
check = x_delta - tot_change

if np.isclose(check, np.zeros(check.shape), atol=1E-6).all():
    print('Decomposition correct')
else:
    print('ERROR - check your decomposition')

Decomposition correct


In [23]:
# create a dataframe where you will store your intermediary results
step = pd.DataFrame([], columns=columns)

## add results to your dataframe
step[step.columns[1]] = tech_change
step[step.columns[2]] = y_change
step[step.columns[3]] = interaction
step[step.columns[0]] = tot_change

## Add the row of total
step.loc["Total"] = step.sum(0)

step.index = [["Eq_8.6"]*4, sectors]

step

Output_Change  Technology_change_contribution  \
Eq_8.6 Sector_1           12.0                        0.784597   
       Sector_2           20.0                        9.660650   
       Sector_3           20.0                        9.963899   
       Total              52.0                       20.409146   

                 Final_Demand_change_contribution  Interaction_term  
Eq_8.6 Sector_1                         11.101581         -0.113822  
       Sector_2                         11.377999          1.038648  
       Sector_3                         10.990034          0.953933  
       Total                            33.469614          1.878759

In [24]:
# Concatenate your results
results = pd.concat([results, step], axis=0)
results

Output_Change  Technology_change_contribution  \
Eq_8.3 Sector_1           12.0                        0.898419   
       Sector_2           20.0                        8.622001   
       Sector_3           20.0                        9.009966   
       Total              52.0                       18.530386   
Eq_8.4 Sector_1           12.0                        0.784597   
       Sector_2           20.0                        9.660650   
       Sector_3           20.0                        9.963899   
       Total              52.0                       20.409146   
Eq_8.5 Sector_1           12.0                        0.898419   
       Sector_2           20.0                        8.622001   
       Sector_3           20.0                        9.009966   
       Total              52.0                       18.530386   
Eq_8.6 Sector_1           12.0                        0.784597   
       Sector_2           20.0                        9.660650   
       Sector_3           20.0                        9.963899   
       Total              52.0                       20.409146   

                 Final_Demand_change_contribution Interaction_term  
Eq_8.3 Sector_1                         11.101581              NaN  
       Sector_2                         11.377999              NaN  
       Sector_3                         10.990034              NaN  
       Total                            33.469614                0  
Eq_8.4 Sector_1                         11.215403              NaN  
       Sector_2                         10.339350              NaN  
       Sector_3                         10.036101              NaN  
       Total                            31.590854                0  
Eq_8.5 Sector_1                         11.215403        -0.113822  
       Sector_2                         10.339350         1.038648  
       Sector_3                         10.036101         0.953933  
       Total                            31.590854         1.878759  
Eq_8.6 Sector_1                         11.101581        -0.113822  
       Sector_2                         11.377999         1.038648  
       Sector_3                         10.990034         0.953933  
       Total                            33.469614         1.878759

#### 1.5.5 Decomposition (eq 8.7)

In [25]:
tech_change = 0.5 * L_delta @ (y_0 + y_1)
y_change = 0.5 * (L_0 + L_1) @ y_delta

In [26]:
# check if our result is complete
tot_change = tech_change + y_change
check = x_delta - tot_change

if np.isclose(check, np.zeros(check.shape), atol=1E-6).all():
    print('Decomposition correct')
else:
    print('ERROR - check your decomposition')

Decomposition correct


In [27]:
# create a dataframe where you will store your intermediary results
step = pd.DataFrame([], columns=columns)

## add results to your dataframe
step[step.columns[1]] = tech_change
step[step.columns[2]] = y_change
step[step.columns[0]] = tot_change

## Add the row of total
step.loc["Total"] = step.sum(0)

step.index = [["Eq_8.7"]*4, sectors]

step

Output_Change  Technology_change_contribution  \
Eq_8.7 Sector_1           12.0                        0.841508   
       Sector_2           20.0                        9.141326   
       Sector_3           20.0                        9.486933   
       Total              52.0                       19.469766   

                 Final_Demand_change_contribution Interaction_term  
Eq_8.7 Sector_1                         11.158492              NaN  
       Sector_2                         10.858674              NaN  
       Sector_3                         10.513067              NaN  
       Total                            32.530234                0

### 1.6 Output table 8.1

In [28]:
# Concatenate your results
results = pd.concat([results, step], axis=0).fillna(0)
results.round(2)

Output_Change  Technology_change_contribution  \
Eq_8.3 Sector_1           12.0                            0.90   
       Sector_2           20.0                            8.62   
       Sector_3           20.0                            9.01   
       Total              52.0                           18.53   
Eq_8.4 Sector_1           12.0                            0.78   
       Sector_2           20.0                            9.66   
       Sector_3           20.0                            9.96   
       Total              52.0                           20.41   
Eq_8.5 Sector_1           12.0                            0.90   
       Sector_2           20.0                            8.62   
       Sector_3           20.0                            9.01   
       Total              52.0                           18.53   
Eq_8.6 Sector_1           12.0                            0.78   
       Sector_2           20.0                            9.66   
       Sector_3           20.0                            9.96   
       Total              52.0                           20.41   
Eq_8.7 Sector_1           12.0                            0.84   
       Sector_2           20.0                            9.14   
       Sector_3           20.0                            9.49   
       Total              52.0                           19.47   

                 Final_Demand_change_contribution  Interaction_term  
Eq_8.3 Sector_1                             11.10              0.00  
       Sector_2                             11.38              0.00  
       Sector_3                             10.99              0.00  
       Total                                33.47              0.00  
Eq_8.4 Sector_1                             11.22              0.00  
       Sector_2                             10.34              0.00  
       Sector_3                             10.04              0.00  
       Total                                31.59              0.00  
Eq_8.5 Sector_1                             11.22             -0.11  
       Sector_2                             10.34              1.04  
       Sector_3                             10.04              0.95  
       Total                                31.59              1.88  
Eq_8.6 Sector_1                             11.10             -0.11  
       Sector_2                             11.38              1.04  
       Sector_3                             10.99              0.95  
       Total                                33.47              1.88  
Eq_8.7 Sector_1                             11.16              0.00  
       Sector_2                             10.86              0.00  
       Sector_3                             10.51              0.00  
       Total                                32.53              0.00